In [1]:
import os

In [3]:
%pwd

'/Users/arpansethi/Documents/ai-projects/kidney-disease/Kidney-Disease-Classification/research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/Users/arpansethi/Documents/ai-projects/kidney-disease/Kidney-Disease-Classification'

In [ ]:
import dagshub
dagshub.init(repo_owner='arpansethi30', repo_name='Kidney-Disease-Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)